In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()

In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code,load_node
from tools import wait_for_node_finished

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home,'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_opt', 
                          computer_local, 
                          '/home/max/Documents/alamode/bin/alm')
code_alm_opt = code
code = get_or_create_code('alamode.anphon', 
                          computer_local, 
                          '/home/max/Documents/alamode/bin/anphon')
code_anphon = code
code = get_or_create_code('alamode.analyze_phonons', 
                          computer_local, 
                          '/home/max/Documents/alamode/bin/analyze_phonons')
code_analyze_phonons = code
code_alm_opt, code_anphon, code_analyze_phonons

In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path

In [ ]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

In [ ]:
g_cwd = nodebank.load("cubic_cwd") # 保存directory
# g_dispfile_folder = nodebank.load("cubic_dispfile_folder")
# g_displace_result = nodebank.load("cubic_displace_result")
g_superstructure = nodebank.load("superstructure") # 長周期構造

g_lammps_all_output = nodebank.load('cubic_all_dfset')
g_cwd, g_lammps_all_output

# extract

In [ ]:
extractworkchain = WorkflowFactory("alamode.extract")

g_format =   nodebank.load("format")
g_superstructure_filepath = nodebank.load("superstructure_filepath")
g_target_file = g_lammps_all_output
g_norder = nodebank.load("cubic_norder")
g_prefix = nodebank.load('cubic_prefix')
g_format, g_norder, g_prefix, g_target_file

In [ ]:
inputs= {'format': g_format,
         'structure_org': g_superstructure_filepath, 
         'target_file': g_target_file,
         'cwd': g_cwd,
         "prefix": g_prefix
        }
print(inputs)

In [ ]:
g_alamodeextract_future = submit(extractworkchain, **inputs)
print(g_alamodeextract_future)
wait_for_node_finished(g_alamodeextract_future)
print(g_alamodeextract_future.is_finished_ok)

# alm opt 

In [ ]:
g_norder

In [ ]:
g_diag = nodebank.load("diag")
g_diag.get_array('diag')

In [ ]:
g_primstructure = nodebank.load("primstructure")
g_primstructure

In [ ]:
g_DFSET_file = g_alamodeextract_future.outputs.dfset_file

g_almprefix = "_".join([g_primstructure.get_formula(),"x".join(map(str,g_diag.get_array("diag"))), "cubic"])
g_almprefix = Str(g_almprefix)

g_wd = nodebank.load_or_dump("cubic_almprefix",g_almprefix) # subdirectory名
g_DFSET_file, g_almprefix, g_wd

In [ ]:
g_fc2xml_file = nodebank.load("anphonfile_xml") # harmonic force constant
print(g_fc2xml_file.filename)

In [ ]:
#codename = "alm_opt@tutor"
#code = Code.get_from_string(codename)
code = code_alm_opt

builder = code.get_builder()
builder.structure = g_superstructure
builder.prefix = g_wd # prefix
builder.cwd = g_cwd # 保存directory
builder.norder = g_norder
builder.dfset_file = g_DFSET_file # displacementとforce
builder.fc2xml_file =  g_fc2xml_file # harmonic term

g_almopt2future = submit(builder)

print(g_almopt2future)
wait_for_node_finished(g_almopt2future, 5)


In [ ]:
g_almopt2future.outputs.result.attributes

## thermal conductivity


In [ ]:
g_primstructure = nodebank.load("primstructure")
g_diag = nodebank.load("diag")
g_qmesh = List(list=[5,5,5])

g_almprefix = "_".join([g_primstructure.get_formula(),
                        "x".join(map(str,g_diag.get_array("diag"))),
                        "harmonic",
                       "q"+"x".join(map(str,g_qmesh.get_list()))
                       ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos, g_cwd

In [ ]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd
builder.cwd = g_cwd
builder.norder = g_norder
builder.fcsxml = g_almopt2future.outputs.input_ANPHON_file
builder.mode = Str("RTA")
#builder.kparam = Dict(dict={"kspacing": 0.1})
builder.qmesh = g_qmesh

key = "RTA"
g_RTAfuture = nodebank.load(key,g_force)
#if g_RTAfuture is None:   
if True:
    g_RTAfuture = submit(builder)
    print(g_RTAfuture)
    wait_for_node_finished(g_RTAfuture, 5)
    if g_RTAfuture.is_finished_ok:
         nodebank.dump(key,g_RTAfuture)

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
from alamode_aiida.data_loader import load_anphon_kl, load_anphon_kl_spec

In [ ]:
g_RTAfuture.outputs.kl_file

In [ ]:
g_df = load_anphon_kl(g_RTAfuture.outputs.kl_file)
g_df.plot(x="Temperature [K]", y="Thermal Conductivity_xx [W/mK]")
import matplotlib.pyplot as plt
plt.xscale("log")

In [ ]:
g_RTAfuture.outputs.result_file

In [ ]:
class analzePhononOptions:
    def __init__(self, **kwargs):
        self.options= {'temp': None, 'mode': None, 'kpoint': None, 
'calc': 'kappa_boundary', 'isotope': None, 'average_gamma': True, 
'size': '1e+6', 'length': None, 'direction': None}

a = analzePhononOptions()
print(a.options)

In [ ]:
#codename = "analyze_phonon@tutor"
#code = Code.get_from_string(codename)
code = code_analyze_phonons

builder = code.get_builder()
builder.cwd = g_cwd
#builder.norder = g_norder
builder.prefix = g_wd
builder.calc = Str("kappa_boundary")
# builder.size = Float(1.0e+6)
builder.file_result = g_RTAfuture.outputs.result_file
builder.param = Dict(dict={"size": 1e+6})

key = "ap_kappa"
g_ap_kappa_future = nodebank.load(key)
if g_ap_kappa_future is None:   
    g_ap_kappa_future = submit(builder)
    print(g_ap_kappa_future)
    wait_for_node_finished(g_ap_kappa_future, 5)
    if g_ap_kappa_future.is_finished_ok:
         nodebank.dump(key,g_ap_kappa_future)

In [ ]:
code = code_analyze_phonons

builder = code.get_builder()
builder.cwd = g_cwd
builder.prefix = g_wd
builder.calc = Str("tau")
# builder.size = Float(1.0e+6)
builder.file_result = g_RTAfuture.outputs.result_file
builder.param = Dict(dict={"temp": 300.0})

key = "ap_tau"
g_ap_tau_future = nodebank.load(key)
if g_ap_tau_future is None:   
    g_ap_tau_future = submit(builder)
    print(g_ap_tau_future)
    wait_for_node_finished(g_ap_tau_future, 5)
    if g_ap_tau_future.is_finished_ok:
         nodebank.dump(key,g_ap_tau_future)

In [ ]:
code = code_analyze_phonons

builder = code.get_builder()
builder.cwd = g_cwd
builder.prefix = g_wd
builder.calc = Str("cumulative")
# builder.size = Float(1.0e+6)
builder.file_result = g_RTAfuture.outputs.result_file
builder.param = Dict(dict={"temp": 300.0, "length": "10000:5"})


key = "ap_cum"
g_ap_cum_future = nodebank.load(key)
if g_ap_cum_future is None:   
#if True:
    g_ap_cum_future = submit(builder)
    print(g_ap_cum_future)
    wait_for_node_finished(g_ap_cum_future, 5)
    if g_ap_cum_future.is_finished_ok:
         nodebank.dump(key,g_ap_cum_future)

In [ ]:
values = g_ap_cum_future.outputs.cumulative.get_array("values")
values

In [ ]:
values = g_ap_cum_future.outputs.cumulative.get_array("values").astype(float)
splitted_header = g_ap_cum_future.outputs.cumulative.get_array("columns")
df = pd.DataFrame(values, columns=splitted_header)
df

In [ ]:
df.plot(x= "L [nm]", y='kappa xx [W/mK]')

## anphon spectra

In [ ]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd
builder.cwd = g_cwd
builder.norder = g_norder
builder.fcsxml = g_almopt2future.outputs.input_ANPHON_file
builder.mode = Str("RTA")
builder.kappa_spec = Int(1) # spectra
builder.qmesh = g_qmesh
eparam = {"EMIN":0, "EMAX":550, "DELTA_E": 10.0}
builder.param= Dict(dict={"general": eparam})

key = "RTAspec"
g_RTAspecfuture = nodebank.load(key,g_force)
#if g_RTAspecfuture is None:  
if True:   
    g_RTAspecfuture = submit(builder)
    print(g_RTAspecfuture)
    wait_for_node_finished(g_RTAspecfuture, 5)
    if g_RTAspecfuture.is_finished_ok:
         nodebank.dump(key,g_RTAspecfuture)


In [ ]:
g_df = load_anphon_kl_spec(g_RTAspecfuture.outputs.kl_spec_file)

In [ ]:
print(np.unique(g_df["Temperature [K]"].values))
g_df_select = g_df[g_df["Temperature [K]"]==300]

In [ ]:
g_df_select.plot(x=g_df.columns[1], y=g_df.columns[2])